# Milestone 3: Implement an Inverted Index and Search

Import spacy and load the small English model.

In [ ]:
import spacy
import json
import os
nlp = spacy.load("en_core_web_sm")

Load the tokenized and processed following TF-IDF data.

In [ ]:
with open("./outputs/milestone-2/tf-idf-data.json") as f:
    data = json.load(f)